In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import pathlib

# Estrazione dati SERIE A stagione 2021-2022

In [3]:
headers = {'User-Agent': 
                'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = "https://fbref.com/it/comp/11/2021-2022/Statistiche-di-Serie-A-2021-2022"
pageTree = requests.get(page)
soup = BeautifulSoup(pageTree.text, features="lxml")

In [4]:
table = soup.select('table.stats_table')[0]
a_hrefs = table.find_all('a')

## Estrazione per una squadra

### Partite di serie A, estrazione match

In [14]:
teams = [a_href for a_href in a_hrefs if '/squadre/' in str(a_href)]        
#for team in teams:
teams.sort(key=lambda x: x.contents[0]) #ordinamento per titolo

team = teams[0]
link = f"https://fbref.com{team.get('href')}"
data = requests.get(link)
match = pd.read_html(data.text, match="Punteggi e partite")[0]
matches_team = match[match['Competizione'] == 'Serie A']

### Estrazione tiri

In [15]:
soup = BeautifulSoup(data.text, features="lxml")
links = soup.find_all('a')

links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

data = requests.get(f"https://fbref.com{links[0]}")

shot = pd.read_html(data.text, match="Tiri")[0]

shot.columns = shot.columns.droplevel()
shots_team = shot[shot['Competizione'] == 'Serie A']

In [ ]:
matches_team.head(2)

In [ ]:
shots_team.head(2)

In [ ]:
dataset = pd.merge(matches_team, shots_team, on = ["Data", "Ora", "Competizione", "Girone", "Giorno", "Stadio", "Risultato", "Rf", "Rs", "Avversario", "xG", "Report partita"])

dataset.to_csv("Season21_22/merge_stats_shots/"+team.contents[0]+".csv")
dataset.head()

## Estrazione dati per tutte le squadre

In [5]:
teams = [a_href for a_href in a_hrefs if '/squadre/' in str(a_href)]        
#for team in teams:
teams.sort(key=lambda x: x.contents[0]) #ordinamento per titolo

all_matches = []

for team in teams:
    team_name = team.contents[0]
    link = f"https://fbref.com{team.get('href')}"
    data = requests.get(link)
    match = pd.read_html(data.text, match="Punteggi e partite")[0]
    matches_team = match[match['Competizione'] == 'Serie A']
    
    soup = BeautifulSoup(data.text, features="lxml")
    links = soup.find_all('a')

    links = [l.get("href") for l in links]
    links = [l for l in links if l and 'all_comps/shooting/' in l]

    data = requests.get(f"https://fbref.com{links[0]}")

    shot = pd.read_html(data.text, match="Tiri")[0]

    shot.columns = shot.columns.droplevel()
    shots_team = shot[shot['Competizione'] == 'Serie A']
    
    merged_dataset = pd.merge(matches_team, shots_team, on = ["Data", "Ora", "Competizione", "Girone", "Giorno", "Stadio", "Risultato", "Rf", "Rs", "Avversario", "xG", "Report partita"])
    merged_dataset['Team'] = team_name

    all_matches.append(merged_dataset)
    
match_df = pd.concat(all_matches)

In [6]:
match_df.columns = [c.lower() for c in match_df.columns]
match_df.to_csv("Season21_22/merge_stats_shots/all_matches.csv")

In [ ]:
match_df

## Estrazione giocatori

In [7]:
team = teams[0]
link = f"https://fbref.com{team.get('href')}"
data = requests.get(link)
players = pd.read_html(data.text, match="Statistiche ordinarie")[0]
players.columns = players.columns.droplevel()
#players

#### Ottengo dati di un singolo giocatore

In [230]:
#for team in teams:
team = teams[0]
team_name = team.contents[0]
link = f"https://fbref.com{team.get('href')}"
data = requests.get(link)
soup = BeautifulSoup(data.text, features="lxml")
links = soup.find_all('a') 
links = [l.get("href") for l in links]
links = [l for l in links if l and '/calciatori/' and '/summary/' in l]
links = list(dict.fromkeys(links)) #elimina duplicati

player_link = f"https://fbref.com{links[0]}"
data = requests.get(player_link)
df_single_player = pd.read_html(data.text, match="Registro partite")[0]

df_single_player.columns = df_single_player.columns.droplevel()
df_single_player_serieA = df_single_player[df_single_player['Competizione']=='Serie A']

#recupero il nome del calciatore
soup = BeautifulSoup(data.text, features="lxml")
player_html_page = soup.select('div.crumbs')[0]
name_player = player_html_page.find_all('strong')[0].get_text()

df_single_player_serieA.insert(0, 'Calciatore', name_player)
df_single_player_serieA.to_csv(name_player+".csv")

### Ottengo i dati per i calciatori di una singola squadra

In [8]:
from tqdm import tqdm

In [12]:
players = []

for team in tqdm(teams[:1]):
    team_name = team.contents[0]
    link = f"https://fbref.com{team.get('href')}"
    data = requests.get(link)
    soup = BeautifulSoup(data.text, features="lxml")
    links = soup.find_all('a') 
    links = [l.get("href") for l in links]
    links = [l for l in links if l and '/calciatori/' and '/summary/' in l]
    links = list(dict.fromkeys(links)) #elimina duplicati
    
    for link in tqdm(links):
        player_link = f"https://fbref.com{link}"
        data = requests.get(player_link)
        df_single_player = pd.read_html(data.text, match="Registro partite")[0]
    
        df_single_player.columns = df_single_player.columns.droplevel()
        df_single_player_team = df_single_player[df_single_player['Competizione']=='Serie A']

        #recupero il nome del calciatore
        soup = BeautifulSoup(data.text, features="lxml")
        player_html_page = soup.select('div.crumbs')[0]
        name_player = player_html_page.find_all('strong')[0].get_text()

        df_single_player_team.insert(0, 'Calciatore', name_player)
        players.append(df_single_player_team)

    team_players = pd.concat(players)
    team_players.columns = [c.lower() for c in team_players.columns]
    team_players.to_csv("Season21_22/players/"+team_name+".csv")



  0%|          | 0/1 [00:10<?, ?it/s]


KeyboardInterrupt: 